In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

2024-05-12 09:01:25.158241: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-12 09:01:25.158408: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-12 09:01:25.389023: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [14]:
# Network and training parameters
EPOCHS = 20
BATCH_SIZE = 128
VERBOSE = 1
NB_CLASSES = 10 #number of outputs = number of digits
N_HIDDEN = 128
VALIDATION_SPLIT = 0.2 # how much train is reserved for VALIDATION
DROPOUT = 0.3

In [15]:
#Loading MNIST dataset
mnist = keras.datasets.mnist
(X_train,Y_train),(X_test,Y_test) = mnist.load_data()
RESHAPED = 784
X_train = X_train.reshape(60000,RESHAPED)
X_test = X_test.reshape(10000,RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
# Normalize inputs to be within in [0,1]
X_train /= 255.0
X_test /= 255.0
print(X_train.shape[0],'train samples')
print(X_test.shape[0],'test samples')
# One-hot representation of the labels
Y_train = tf.keras.utils.to_categorical(Y_train,NB_CLASSES)
Y_test = tf.keras.utils.to_categorical(Y_test,NB_CLASSES)

60000 train samples
10000 test samples


In [14]:
# Build the model
# Variation-1
# model = tf.keras.models.Sequential()
# model.add(keras.layers.Dense(NB_CLASSES,
#                             input_shape=(RESHAPED,),
#                             name='dense_layer',
#                             activation='softmax'))

In [4]:
# Variation-2
# model = tf.keras.models.Sequential()
# model.add(keras.layers.Dense(N_HIDDEN,
#                             input_shape=(RESHAPED,),
#                             name='dense_layer',
#                             activation='relu'))
# model.add(keras.layers.Dense(N_HIDDEN,
#                             name='dense_layer_2',
#                             activation='relu'))

# model.add(keras.layers.Dense(NB_CLASSES,
#                             name='dense_layer_3',
#                             activation='softmax'))
# # summary of the model
# model.summary()

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_layer (Dense)             │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer_2 (Dense)           │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer_3 (Dense)           │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,282 (462.04 KB)

 Trainable params: 118,282 (462.04 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
# Variation-3
model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(N_HIDDEN,
                            input_shape=(RESHAPED,),
                            name='dense_layer',
                            activation='relu'))

model.add(keras.layers.Dropout(DROPOUT))

model.add(keras.layers.Dense(N_HIDDEN,
                            name='dense_layer_2',
                            activation='relu'))

model.add(keras.layers.Dropout(DROPOUT))

model.add(keras.layers.Dense(NB_CLASSES,
                            name='dense_layer_3',
                            activation='softmax'))
# summary of the model
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_layer (Dense)             │ (None, 128)            │       100,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer_2 (Dense)           │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_layer_3 (Dense)           │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 118,282 (462.04 KB)

 Trainable params: 118,282 (462.04 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
# Compiling the model.
# SGD Optimizer
# model.compile(optimizer='SGD',loss='categorical_crossentropy',
#              metrics=['accuracy'])

# Adam Optimizer
model.compile(optimizer='Adam',loss='categorical_crossentropy',
             metrics=['accuracy'])

In [18]:
# Training the model.
model.fit(X_train,Y_train,batch_size=BATCH_SIZE,
         epochs=EPOCHS,verbose=VERBOSE,
          validation_split=VALIDATION_SPLIT)

Epoch 1/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7219 - loss: 0.8822 - val_accuracy: 0.9455 - val_loss: 0.1851
Epoch 2/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9284 - loss: 0.2389 - val_accuracy: 0.9600 - val_loss: 0.1355
Epoch 3/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9455 - loss: 0.1852 - val_accuracy: 0.9638 - val_loss: 0.1211
Epoch 4/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.9533 - loss: 0.1523 - val_accuracy: 0.9693 - val_loss: 0.1039
Epoch 5/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9613 - loss: 0.1294 - val_accuracy: 0.9694 - val_loss: 0.1022
Epoch 6/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9652 - loss: 0.1134 - val_accuracy: 0.9708 - val_loss: 0.0944
Epoch 7/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9671 - loss: 0.1068 - val_accuracy: 0.9737 - val_loss: 0.0872
Epoch 8/20
375/375 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9708 - loss: 0.0943 - val_accuracy: 0.

In [19]:
# Evaluate the model
test_loss, test_acc = model.evaluate(X_test,Y_test)
print('Test accuracy:',test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9753 - loss: 0.0848
Test accuracy: 0.9786999821662903
